In [ ]:
import os

import numpy as np
import pandas as pd

from apple import *
from mesa import *

### Apple Watch Data

In [ ]:
PATH = '../data/watch/processed/cropped/'
# PATH = '../data/watch/origin/'

In [ ]:
subject_ids = []
for n, filename in enumerate(os.listdir(PATH)):
    filename = filename.split('_')
    subject_id = int(filename[0])

    if subject_id not in subject_ids:
        subject_ids.append(subject_id)
    
# sort the list
sorted_subject_ids = sorted(subject_ids)

In [ ]:
np.array(sorted_subject_ids)

In [ ]:
apple_data = generate_data_to_dic(PATH,
                                  second_column_step=1,val_to_fill_nans=-100,
                                  print_analysis=False)


for subj in sorted_subject_ids:
    savepath = "../data/watch/processed/temp/subject_" + str(subj) + ".csv"
    apple_data[subj].to_csv(savepath, index=False)


df, psg_status = filter_to_epoch("../data/watch/processed/temp/", bin_size=30)


for subj in sorted_subject_ids:
    ses_id = psg_status[subj].keys()
    last_ses = list(ses_id)[-1]
    stage = psg_status[subj].values()
    
    temp = pd.DataFrame({'session_id': ses_id, 'new_psg_status': stage})
    
    new_df = pd.merge(df[subj], temp, on='session_id')
    new_df.drop(columns=["psg_status"], inplace=True)
    
    new_df.rename(columns={'new_psg_status': 'psg_status'}, inplace=True)
    
    last_info = new_df[new_df['session_id'] == last_ses]
    
    if len(last_info) != 30:
        new_df.drop(new_df[new_df['session_id'] == last_ses].index, inplace=True)
    
    savepath = "../data/watch/processed/final/subject_" + str(subj) + ".csv"
    new_df.to_csv(savepath, index=False)

In [ ]:
new_df

### MESA Data

In [ ]:
import os

import numpy as np
import pandas as pd

from mesa import *

In [ ]:
DATAPATH = '/volumes/990Pro_2TB/UofT/Intro_DL/project/data/'
# DATAPATH = '/NFS'
subject_ids = get_subject_ids(PATH=os.path.join(DATAPATH, "./mesa/polysomnography/annotations-events-nsrr"))
len(subject_ids)

In [ ]:
SAVEPATH="/NFS/Users/moonsh/data/mesa/preproc/npy/"
Error = mesa_preprocessing(subject_ids, savepath=SAVEPATH)

In [ ]:
for subj in subject_ids:

    if subj not in Error:
        activity = np.load(SAVEPATH+subj+"_activity_count.npy").reshape(-1)
        hr = np.load(SAVEPATH+subj+"_heart_rate.npy").reshape(-1)
        ecg = np.load(SAVEPATH+subj+"_ecg.npy").reshape(-1)
        psg_status = np.load(SAVEPATH+subj+"_labeled_sleep.npy").squeeze()
        repeat_psg = []
        repeat_cosine = []
        session = []
        session_ecg = []
        psg_ecg = []

        ses = 0

        for i, psg in enumerate(psg_status):
            repeat_psg.extend([psg]*30)            
            session.extend([ses]*30)
            session_ecg.extend([ses]*256)
            psg_ecg.extend([psg]*256)
            
            ses += 1

        print(activity.shape, hr.shape, repeat_cosine.shape, len(repeat_psg), len(psg_ecg), len(session_ecg))

        df = pd.DataFrame({'activity_count': activity, 'heart_rate': hr, 'session_id':session, 'psg_status': repeat_psg})
        df_ecg = pd.DataFrame({"ECG":ecg, 'session_id':session_ecg, 'psg_status': psg_ecg})
        
        df.to_csv(f"/NFS/Users/moonsh/data/mesa/preproc/final/subject_{subj}.csv", index=False)
        df_ecg.to_csv(f"/NFS/Users/moonsh/data/mesa/preproc/final/subject_{subj}_ecg.csv", index=False)

In [ ]:
# error_df = pd.DataFrame({'subject_id': Error})
# error_df.to_csv("../data/mesa/error.csv", index=False)